In [1]:
import pandas as pd
import numpy as np
INPUT_FILE = 'data\\den_a_tochka.csv'
OUTPUT_RABOTA = 'cleared\\rabota2022.csv'
OUTPUT_ZAKAZ = 'cleared\\zakaz2022.csv'
data = pd.read_csv(INPUT_FILE, encoding = 'cp1251', sep=';')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5405 entries, 0 to 5404
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Наименование работ            5405 non-null   object 
 1   Unnamed: 1                    3858 non-null   object 
 2   Unnamed: 2                    2353 non-null   object 
 3   Unnamed: 3                    3858 non-null   object 
 4   Unnamed: 4                    2353 non-null   object 
 5   Unnamed: 5                    3858 non-null   object 
 6   Unnamed: 6                    0 non-null      float64
 7   Unnamed: 7                    0 non-null      float64
 8   Unnamed: 8                    0 non-null      float64
 9   Unnamed: 9                    0 non-null      float64
 10  Unnamed: 10                   0 non-null      float64
 11  Unnamed: 11                   0 non-null      float64
 12  Unnamed: 12                   0 non-null      float64
 13  Объ

In [2]:
for col in data.columns:
    if data[col].isna().sum() == data.shape[0]:
        data.drop(columns=col, inplace=True)

drop_cols = [
    'Расценка(тариф)', 
    'КТУ',
    'Сумма',
    'Объемработ сучетомКТУ(н/час)',
    'Unnamed: 3',
    'Unnamed: 2',
    'Unnamed: 5'
    ]
data.drop(columns=drop_cols, inplace=True)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5405 entries, 0 to 5404
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Наименование работ  5405 non-null   object
 1   Unnamed: 1          3858 non-null   object
 2   Unnamed: 4          2353 non-null   object
 3   Объемработ(н/час)   30 non-null     object
dtypes: object(4)
memory usage: 169.0+ KB


In [4]:
# Общий итог
print(data['Unnamed: 1'].iloc[-1]) 
# Убираем его
data = data.iloc[:-1]

3 050 929,000


In [5]:
data.fillna('0', inplace=True)
data['Цена'] = data['Unnamed: 1'].apply(lambda x: int(x.replace('\xa0','').split(',')[0])) + data['Объемработ(н/час)'].apply(lambda x: int(x.replace('\xa0','').split(',')[0]))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5404 entries, 0 to 5403
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Наименование работ  5404 non-null   object
 1   Unnamed: 1          5404 non-null   object
 2   Unnamed: 4          5404 non-null   object
 3   Объемработ(н/час)   5404 non-null   object
 4   Цена                5404 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 211.2+ KB


In [6]:
drop_cols = ['Unnamed: 1', 'Unnamed: 4', 'Объемработ(н/час)']
data.drop(columns=drop_cols, inplace=True)
data

,Наименование работ,Цена
0,"Заказ-наряд №: 11 от 30.07.2021, ТС: Hyundai i...",0
1,"Заправка кондиц.(газ,масло)и экспресс диагност...",1700
2,Итого,1700
3,"Заказ-наряд №: 16 от 31.07.2021, ТС: ВАЗ 2109 ...",0
4,"Ведомая диагностика поиска неисправностей ,чте...",1000
...,...,...
5399,Электротехнические работы,2000
5400,Итого,2000
5401,"Заказ-наряд №: 8717 от 19.02.2023, ТС: Chevrol...",0
5402,Адаптация,500


In [7]:
data['Дата'] = pd.to_datetime(data['Наименование работ'].apply(lambda x: x[x.find('от')+3:x.find(',')] if 'Заказ-наряд №' in x else np.nan), dayfirst=True)
data['Дата'].fillna(method='ffill', inplace=True)

In [8]:
# Делаем датафрейм по нименования работ
df_rabota = data[data['Наименование работ'].apply(lambda x: (x != 'Итого') and ('Заказ-наряд №' not in x))]
df_rabota.reset_index(drop=True, inplace=True)
df_rabota

,Наименование работ,Цена,Дата
0,"Заправка кондиц.(газ,масло)и экспресс диагност...",1700,2021-07-30
1,"Ведомая диагностика поиска неисправностей ,чте...",1000,2021-07-31
2,"Ведомая диагностика поиска неисправностей ,чте...",1000,2021-07-31
3,Ремонт генератора,1000,2021-07-31
4,Электротехнические работы прошивка системы,8000,2021-07-31
...,...,...,...
2365,Ремонт(восстановление)электропроводки фар с за...,1500,2023-02-15
2366,Диагностика электрооборудования,500,2023-02-16
2367,Проверка ламп,300,2023-02-17
2368,Электротехнические работы,2000,2023-02-18


In [9]:
# Делаем датафрейм заказ-нарядов
itogi = data['Цена'].to_frame()[data['Наименование работ'].apply(lambda x: x == 'Итого')]
itogi.reset_index(drop=True, inplace=True)
zakaz = data[['Наименование работ', 'Дата']][data['Наименование работ'].apply(lambda x: 'Заказ-наряд №' in x)]
zakaz.reset_index(drop=True, inplace=True)
df_zakaz = pd.concat([zakaz, itogi], axis=1) 
df_zakaz.rename(columns={'Наименование работ': 'Заказ наряд'}, inplace=True)

df_zakaz['ТС'] = df_zakaz['Заказ наряд'].apply(lambda x: x[x.find('ТС:')+4:x.find('гос.')-1])
df_zakaz['Гос номер'] = df_zakaz['Заказ наряд'].apply(lambda x: x[x.find('номер:')+7:])
df_zakaz

,Заказ наряд,Дата,Цена,ТС,Гос номер
0,"Заказ-наряд №: 11 от 30.07.2021, ТС: Hyundai i...",2021-07-30,1700,Hyundai ix35,А681НС196
1,"Заказ-наряд №: 16 от 31.07.2021, ТС: ВАЗ 2109 ...",2021-07-31,1000,ВАЗ 2109,Т326ЕОЛ72
2,"Заказ-наряд №: 18 от 31.07.2021, ТС: ВАЗ 2114 ...",2021-07-31,1000,ВАЗ 2114,В034НС72
3,"Заказ-наряд №: 19 от 31.07.2021, ТС: Toyota Ca...",2021-07-31,1000,Toyota Camry,Х050АТ96
4,"Заказ-наряд №: 20 от 31.07.2021, ТС: Hyundai S...",2021-07-31,8000,Hyundai Solaris,О564КН72
...,...,...,...,...,...
1512,"Заказ-наряд №: 8686 от 15.02.2023, ТС: Kia Rio...",2023-02-15,1500,Kia Rio,С012УХ72
1513,"Заказ-наряд №: 8689 от 16.02.2023, ТС: LADA VE...",2023-02-16,500,LADA VESTA,Т791РМ72
1514,"Заказ-наряд №: 8704 от 17.02.2023, ТС: Самолет...",2023-02-17,300,Самолето Ветролет,
1515,"Заказ-наряд №: 8714 от 18.02.2023, ТС: Ravon R...",2023-02-18,2000,Ravon R4,У525КН72


In [10]:
df_zakaz.to_csv(OUTPUT_ZAKAZ)
df_rabota.to_csv(OUTPUT_RABOTA)